# #411 Multi span post-tensioned concrete tee beam and slab bridge deck with pier columns
<i>Deck slab modelled in shell elements, tee beams and piers modelled with beam elements. Influence analysis based on sliced resultants</i>
***

In [ ]:
solve_analyses = False
dmi_grid = False # Use nodes in search area instead

Define model parameters

In [ ]:
# Span lengths
span_lengths = [25, 35, 25]
contraflexure = 0.15
skew_angle = 25

# Deck 
beam_spacing = [3, 3, 3, 3, 3]

deck_overhang = 1.5
deck_thk = 0.2
deck_thk_outstand = 0.25
# deck_thk_pier = 1.4
tee_depth = 1.0
tee_web_thk = 0.5

# Piers
column_dia = 1.1
pier_head = 2.0
abut_width = 1.0
pier_height = 5

# Meshing
mesh_size = 2

# Loading area
carriageway_width = 11

# Loads
surf_load_intensity = -2.5
parapet_load_udl = -2.5
footpath_surface_load_intensity = -1 # Additional thickness of footpath
settlement = -0.01 # m

# Temperatures
temperature_increase = 20
temperature_decrease = -20
temp_max = 25
temp_min = 0.0

# Live loads
pedestrian_live_load = -5 # kpa
breaking_load = 900 # kN
horiz_load = 200    # kN
pier_impact_load = 1000 # kN
deck_impact_load = 1000 # kN/m

In [ ]:
# Calculated dimensions
overall_length = sum(span_lengths) + abut_width
overall_width  = sum(beam_spacing) + 2 * deck_overhang
footway_width  = (overall_width-carriageway_width)/2

breaking_load_intensity = breaking_load / (carriageway_width*overall_length)

# eccentcity_pier = (deck_thk_pier-deck_thk)/2
eccentcity_outstand = (deck_thk_outstand-deck_thk)/2
pier_head_gap = deck_thk/2 + tee_depth

Connect to LUSAS and either create a new model or delete the contents of the existing one - be careful if you have a model already loaded

In [ ]:
import numpy as np
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

# Create a new model
lusas.newProject("Structural", f"Post Tensioned Tee beam Bridge Deck")
# Reference to the model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [ ]:
# Main tee beam element mesh
beam_mesh_attr = db.createMeshLine("Tee beam mesh").setSize("BMI21", mesh_size)

# Tee section geometric attribute
beam_geom_attr = Helpers.create_rectangular_section(db, 'Tee section', tee_web_thk, tee_depth)
beam_geom_attr.setValue("ez", tee_depth / 2 + deck_thk / 2)

In [ ]:
# Pier cross heads
pier_head_mesh_attr = db.createMeshLine("Pier head mesh").setSize("BMI21", mesh_size)
# Pier head geometric attribute
pier_head_geom_attr = Helpers.create_rectangular_section(db, 'Pier head', 1, 1)

In [ ]:
# Pier columns
pier_col_mesh_attr = db.createMeshLine("Pier column mesh").setSize("BMI21", mesh_size)
# Set the upper section which overlaps with the deck to be rigid
pier_col_mesh_attr.setEndRigidZoneSameAsStart(False).setRigid("Start", "formulaNoError", False, 1.0, 1.0)
# Pier column geometric attribute
pier_geom_attr = Helpers.create_circular_section(db, 'Pier column', column_dia)

In [ ]:
# End diaphragms
diaphragm_mesh_attr = db.createMeshLine("End diaphragm mesh").setSize("BMI21", mesh_size)
# Diaphragm section geometric attribute
diaphragm_geom_attr = Helpers.create_rectangular_section(db, 'Diaphragm section', 0.5, 1.0)
diaphragm_geom_attr.setValue("ez", 1 / 2 + deck_thk / 2)

In [ ]:
# Deck mesh attribute, thick shell elements
deck_mesh_attr    = db.createMeshSurface("SMsh1").setRegularSize("QTS4", mesh_size, True)
# Deck surface thickness attribute
deck_surface_attr      = db.createGeometricSurface("Deck Slab").setSurface(deck_thk, 0.0)
# pier_head_surface_attr = db.createGeometricSurface("Deck Slab at Pier").setSurface(deck_thk_pier, eccentcity_pier)
outstand_surface_attr  = db.createGeometricSurface("Deck Outstand").setSurface(deck_thk_outstand, eccentcity_outstand)

In [ ]:
# Bearing joints between pier heads and longitudinal beams
joint_mesh_attr = db.createPointMeshElementAttr("Bearing Joint", "JNT4")

joint_material_attr = db.createSpringJointMaterial("Bearing Material", [10e9, 10e9, 10e9])
joint_material_attr.setValue("Assignment", "Point")

In [ ]:
# Concrete material attribute
concrete_mat_attr = db.createIsotropicMaterial("Concrete", 34.8E6, 0.2, 2.4, 10e-6)

# Set the Concrete material to be default and automatically assigned to all newly created geometry 
db.setAsDefault("Material", concrete_mat_attr)

In [ ]:
# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")
support_fixed_attr  = db.createSupportStructural("Fully Fixed").setStructural("R", "R", "R", "R", "R", "R", "F", "F", "C", "F")

## Create the model geometry

In [ ]:
# Helper function to calculate beam coordinates accounting for the skew angle
def skew_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

In [ ]:
# X Coordinates for the entire deck grid
x_grid = list()
x = abut_width/2
# First support
x_grid.extend([0.0, x, x+abut_width/2])
# Pier lines
for iy in range(0, len(span_lengths)-1):
    x+=span_lengths[iy]
    x_grid.extend([x-span_lengths[iy]*contraflexure, x, x+span_lengths[iy+1]*contraflexure])
# final support
x+=span_lengths[-1]
x_grid.extend([x-abut_width/2, x, x+abut_width/2])

# Y Coordinates for deck grid
s1 = []
for s in [ s/2 for s in beam_spacing ]:
    s1.append(s)
    s1.append(s)

spacing = [deck_overhang] + s1 + [deck_overhang]
y_grid = [0]+list(np.add.accumulate(spacing))
y_grid = [i - overall_width/2 for i in y_grid]


In [ ]:
# Create the actual points in the model [Y][X]
deck_points = [[Helpers.create_point(x+skew_x(y), y, 0.0) for x in x_grid] for y in y_grid]

In [ ]:
''' Create Deck surfaces '''
# Set the default mesh attribute which is automatically assigned to each newly created surface
db.setAsDefault("Surface Mesh", deck_mesh_attr)

# Create the surfaces for the first outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    Helpers.create_surface_from_points([ deck_points[0][ix], deck_points[0][ix+1], deck_points[1][ix+1], deck_points[1][ix] ])

# Create the surfaces for the main deck
db.setAsDefault("Surface Geometric", deck_surface_attr)
for iy in range(1, len(y_grid)-2):
    for ix in range(0, len(x_grid)-1):
        Helpers.create_surface_from_points([ deck_points[iy][ix], deck_points[iy][ix+1], deck_points[iy+1][ix+1], deck_points[iy+1][ix] ])

# Create the surfaces for the second outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    Helpers.create_surface_from_points([ deck_points[-2][ix], deck_points[-2][ix+1], deck_points[-1][ix+1], deck_points[-1][ix] ])


In [ ]:
beam_indicies = []
for iy in range(1, len(y_grid)-1):
    if iy % 2 == 1:
        beam_indicies.append(iy)

In [ ]:
# Beam shell slcing parameters
slice_type = 4      # Constant spacing
slice_distances = "2"
include_points = False
start_distance = 0
about_neutral_axis = True
use_effective_width = False         # Use groups instead
effective_width = 0.0
whole_elements = False
smooth_corners = True
extent = 2 # Group
rotation = 0

In [ ]:
# Beam mesh
girder_no = 1
for no, iy in enumerate(beam_indicies):
    girder_no = no + 1

    line = Helpers.get_line_between_points(deck_points[iy][0], deck_points[iy][1])
    group = db.createGroup(f"Girder {girder_no}")
    group.add(line)
    group.addColinearNeighbours(1)
    beam_mesh_attr.assignTo(group)
    beam_geom_attr.assignTo(group)

    # now add the adjacent deck surfaces so the group contains all contributing elements for the slicing
    beam_lines = ','.join([str(l.getID()) for l in group.getObjects("Line")])
    group.addHOF("Surface")
    slice = db.createBeamShellSlice(f"Girder {girder_no}")
    slice.setBeamShellSlice(slice_type, slice_distances, include_points, start_distance, about_neutral_axis, use_effective_width, 
                            effective_width, whole_elements, smooth_corners, "Slice", extent, group.getName(), beam_lines, rotation)

lusas.enableTrees(True) #needed to refresh the treeview and display the beam shell slices

In [ ]:
# Determine the array indices for all support points
support_indices = [1]
for s in span_lengths:
    support_indices.append(support_indices[-1]+3)

In [ ]:
# Sub structures and supports

# List of List of supported points, we'll use this later to assign displacements in the settlement analysis
all_support_points = list()
# List of pier lines, we'll use this later to assign impact loads
pier_column_lines = list()

# Pier indices
pier_indices_X = support_indices[1:-1]
pier_indices_Y = [2]
for i in range(len(beam_spacing)-1):
    pier_indices_Y.append(pier_indices_Y[-1]+2)

# bearing joints requiore coincident points, make sure they dont merge together
db.options().setBoolean("newFeaturesMergeable", False)

rigid_link_id = 1

# Create pier lines
for ix in pier_indices_X:
    lusas.assignment().setAllDefaults()
    # Create a pier head
    pier_points = []
    for iy in range(len(y_grid)):
        p = deck_points[iy][ix]
        pier_points.append(Helpers.create_point(p.getX(), p.getY(), p.getZ() - pier_head_gap))

    for i in range(len(pier_points)-1):
        line = Helpers.create_line_from_points(pier_points[i], pier_points[i+1])
        pier_head_mesh_attr.assignTo(line)
        pier_head_geom_attr.assignTo(line)

    bearing_points = []
    for i in beam_indicies:
        p = pier_points[i]
        bearing_points.append(Helpers.create_point(p.getX(), p.getY(), p.getZ()))

        objs = lusas.newObjectSet().add(p).add(bearing_points[-1])
        joint_mesh_attr.assignTo(objs)
        joint_material_attr.assignTo(p)

        # Form a rigid link between the top of the bearing and the point in th edeck plane
        rigid_link_attr = db.createRigidLinkConstraint(f"Rigid Link {rigid_link_id}")
        objs = lusas.newObjectSet().add(bearing_points[-1]).add(deck_points[i][ix])
        rigid_link_attr.assignTo(objs)
        rigid_link_id+=1
    

    supports = list()
    for iy in pier_indices_Y:
        deck_point = pier_points[iy]
        ground_point = Helpers.create_point(deck_point.getX(), deck_point.getY(), -pier_height)
        pier_column_lines.append(Helpers.create_line_from_points(deck_point, ground_point))
        # Assign the fixed support attribute to the bottom point
        support_fixed_attr.assignTo(ground_point)
        supports.append(ground_point)
    all_support_points.append(supports)
    pier_geom_attr.assignTo(pier_column_lines)
    pier_col_mesh_attr.assignTo(pier_column_lines)

db.options().setBoolean("newFeaturesMergeable", True)


In [ ]:
# End diaphragms
for ix in [support_indices[0], support_indices[-1]]:
    for iy in range(len(y_grid)-1):
        l = Helpers.get_line_between_points(deck_points[iy][ix], deck_points[iy+1][ix])
        diaphragm_mesh_attr.assignTo(l)
        diaphragm_geom_attr.assignTo(l)

In [ ]:
# End 1 slide supports
all_support_points.insert(0, list())
for iy in beam_indicies:
    deck_point = deck_points[iy][1]
    support_slide_attr.assignTo(deck_point)
    all_support_points[0].append(deck_point)

# End 2 slide supports
all_support_points.append(list())
for iy in beam_indicies:
    deck_point = deck_points[iy][len(x_grid) - 2]
    support_slide_attr.assignTo(deck_point)
    all_support_points[-1].append(deck_point)

In [ ]:
# Turn off default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Surface Mesh", "None")
db.setAsDefault("Line Geometric", "None")
db.setAsDefault("Surface Geometric", "None")
db.setAsDefault("Isotropic Material", "None")

Generate the mesh

In [ ]:
db.resetMesh()
db.updateMesh()

Define the carriageway by two kerb lines

In [ ]:
overall_length = sum(span_lengths) + abut_width
# Create lines to represent the carriageway kerbs that can be used for the Vehicle Load Optimiser
curb_line1 = Helpers.create_line([-10,  carriageway_width/2, 0.5], [10 + overall_length,  carriageway_width/2, 0.5])
curb_line2 = Helpers.create_line([-10, -carriageway_width/2, 0.5], [10 + overall_length, -carriageway_width/2, 0.5])
# Set the colour to distinguish them from model lines
curb_line1.setPen(2)
curb_line2.setPen(2)

# Loads

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)

In [ ]:
# Create a loadcase for the Parapet loads to be applied to the entire deck surface
parapet_loadcase = db.createLoadcase("Parapet Loads")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(parapet_loadcase)
# Create a locally distributed load for Parapet loads
parapet_load_attr = db.createLoadingGlobalDistributed("Parapet Loads").setGlobalDistributed("Length", 0.0, 0.0, parapet_load_udl, 0.0, 0.0, 0.0)
# and assign to all the edge lines

# Start with the first point at either edge of the deck
for iy in [0, len(y_grid)-1]:
    p0 = deck_points[iy][0]
    p1 = deck_points[iy][1]
    # Find the line that connects the first and second points, i.e. along the x axis and add to the object set
    objs = lusas.newObjectSet()
    for geom in p0.getHOFs():
        if geom.getEndPoint() == p1 or geom.getStartPoint() == p1: # Line directions are different so check both points
            objs.add(geom)
    # Include all connected collinear lines
    objs.addColinearNeighbours(1)
    # Assign the parapet loads
    parapet_load_attr.assignTo(objs, assignment)

In [ ]:
# Create a loadcase for the Surfacing loads to be applied to the entire deck surface
surfacing_loadcase = db.createLoadcase("Surfacing")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(surfacing_loadcase)
# Create a locally distributed load for Surfacing loads
surf_load_attr = db.createLoadingLocalDistributed("Surfacing").setLocalDistributed(0.0, 0.0, surf_load_intensity, "surface")
# and assign to all the surfaces
surf_load_attr.assignTo("Surfaces", assignment)

Discrete loads can be be re-used in the model and assigned to a different origin. We'll do that for the footpath loads.<br/>
We'll make a generic function to create a 4 noded discrete patch load the size of the footpath.

In [ ]:
POINT_OFFSET = 9 # Points created for assignment of discrete loads will be offset from the deck for clarity
Z_OFFSET = 1

def create_footpath_load(name:str, load1:float, load2:float)->'IFLoadingDiscretePatch':
    attr = db.createLoadingDiscretePatch(name).setDiscretePatch("surf4", "Z")
    # Add the points of an 4 node patch load in which the load intensity varies across the width
    # The x coordinates are increased by 1 because we'll assign these loads to points created at x=-1 to hold these loads
    attr.addRow(POINT_OFFSET,                0.0,           Z_OFFSET, load1)
    attr.addRow(POINT_OFFSET,                footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, 0.0,           Z_OFFSET, load1)
    # Explicitly specify how the patch load should be divided up.
    # We know the width of the footway is small so we dont need so many divisions
    # Limiting the divisions speeds up the time to resolve the patch load into nodal loads
    attr.setDivisions(math.ceil(footway_width), 40)

    return attr

In [ ]:
# Create a surfacing load using the generic function
footpath_surf_load_attr = create_footpath_load("Footpath Surface", footpath_surface_load_intensity, footpath_surface_load_intensity)

# Assign this to a new point using the assignment object set up for the deck surfacing loads
footpath_point1 = Helpers.create_point(-POINT_OFFSET, -overall_width/2, 0.0)
footpath_point1.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point1, assignment)
# And again to a 2nd new point
footpath_point2 = Helpers.create_point(-POINT_OFFSET, carriageway_width/2, 0.0)
footpath_point2.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point2, assignment)

# Note that we set the colour of the new points to blue so we can identify that they are not part of the model and only used to locate the discrete loads

Create a settlement load acting only on each support

In [ ]:
# Create the prescribed displacement attribute to represent vertical settlement
settlement_attr = db.createPrescribedDisplacementLoad("Settlement", "Total")
settlement_attr.setDisplacement("W", settlement)

# Create a loadcase for each support settlement
for iy in range(0, len(all_support_points)):
    settlement_loadcase = db.createLoadcase(f"Settlement {iy+1}")
    # Set the loadcase in the assignment object
    assignment = lusas.assignment().setAllDefaults().setLoadset(settlement_loadcase)
    # Use the surface set defined early to assign the load to all deck surfaces
    for p in all_support_points[iy]:
        settlement_attr.assignTo(p, assignment)

# Variable Loads

Create temperature loads

In [ ]:
# Create a loadcases for temperature
temp_warm_loadcase = db.createLoadcase("Temperature +dT")
temp_cool_loadcase = db.createLoadcase("Temperature -dT")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)

In [ ]:
# Create a loadcases for temperature Gradient
temp_warm_grad_loadcase = db.createLoadcase("Temperature +dT Grad")
temp_cool_grad_loadcase = db.createLoadcase("Temperature -dT Grad")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperatureProfile("Warming Gradient").setTopTemperature(temp_max).setBottomTemperature(temp_min)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperatureProfile("Cooling Gradient").setTopTemperature(temp_min).setBottomTemperature(temp_max)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_grad_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_grad_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)

Pedestrian live loads

In [ ]:
# We can reuse the discrete patch load function to create pedestrian live loads 
ped_live_load_attr = create_footpath_load("Pedestrian Live load", pedestrian_live_load, pedestrian_live_load)

# Create a loadcase for the pedestrian live loads
ped_loads_loadcase = db.createLoadcase("Pedestrian live loads")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(ped_loads_loadcase)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Again, assign the same load to the two dedicated points
ped_live_load_attr.assignTo(footpath_point1, assignment)
ped_live_load_attr.assignTo(footpath_point2, assignment)

Breaking load

In [ ]:
# Create a discrete patch load with the loads acting in the X direction.
# Patch loads are projected onto the structure in the direction of their normal. Which will be the Z axis in this case.
breaking_load_attr = db.createLoadingDiscretePatch("Braking load 1").setDiscretePatch("surf4", "X")
# Add the points of an 4 node patch load
breaking_load_attr.addRow(POINT_OFFSET,                -carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET,                 carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length,  carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length, -carriageway_width/2, Z_OFFSET, breaking_load_intensity)

# Create a loadcase for the braking loads
breaking_loads_loadcase1 = db.createLoadcase("Braking load +ve")
breaking_loads_loadcase2 = db.createLoadcase("Braking load -ve")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(breaking_loads_loadcase1)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Assign to the point at the origin
point0 = Helpers.create_point(-POINT_OFFSET, 0.0, 0.0)
point0.setPen(2)
breaking_load_attr.assignTo(point0, assignment)

# Assign the same load with a negative load factor for the opposite direction
assignment.setLoadFactor(-1)
assignment.setLoadset(breaking_loads_loadcase2)
breaking_load_attr.assignTo(point0, assignment)

Horizontal loads

In [ ]:
# Create the concentrated load attribute
horiz_load_pos_attr = db.createLoadingConcentrated("Horizontal +ve")
horiz_load_pos_attr.setConcentrated(0.0, horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
horiz_load_neg_attr = db.createLoadingConcentrated("Horizontal -ve")
horiz_load_neg_attr.setConcentrated(0.0, -horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

# Assign positive and negative horizontal loads to each pier
count = 1
for ix in pier_indices_X:
    horiz_pos_loadcase = db.createLoadcase(f"Horizontal +ve {count}")
    horiz_neg_loadcase = db.createLoadcase(f"Horizontal -ve {count}")
    # Assign the lateral load to the central column of each pier
    deck_point = deck_points[pier_indices_Y[1]][ix]
    assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_pos_loadcase)
    horiz_load_pos_attr.assignTo(deck_point, assignment)
    assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_neg_loadcase)
    horiz_load_neg_attr.assignTo(deck_point, assignment)
    count+=1

# Impact Loads

In [ ]:
# Impact load defined as a point load at mid height of the pier leg
impact_load_attr = db.createLoadingBeamPoint("Pier Impact").setBeamPoint("Parametric", "Global", "beam")
impact_load_attr.addRow(0.5, -pier_impact_load/2, pier_impact_load, 0.0, 0.0, 0.0, 0.0)
# Create the loadcase - we'll just consider a single case here
impact_loadcase = db.createLoadcase(f"Pier Impact")
# Assign to the first pier leg
impact_load_attr.assignTo(pier_column_lines[0], assignment.setAllDefaults().setLoadset(impact_loadcase))

In [ ]:
# Impact load applied to the deck
# Create a discrete line load acting in the global Y direction, but projected onto the model in the Z direction
deck_impact_load_attr = db.createLoadingDiscretePatch("Deck Impact").setDiscretePatch("line2", "Y", [0,0,1])
# Specify the load position relative to the 
deck_impact_load_attr.addRow(span_lengths[1]/2 -0.5, 0.0, 0.0, deck_impact_load)
deck_impact_load_attr.addRow(span_lengths[1]/2 +0.5, 0.0, 0.0, deck_impact_load)
# Create the loadcase
impact_loadcase = db.createLoadcase(f"Deck Impact")
# Assign to the point at the start of span 2
p0 = deck_points[1][4]
deck_impact_load_attr.assignTo(p0, assignment.setAllDefaults().setLoadset(impact_loadcase))

# Load Groups

In [ ]:
''' Create some intermediate load groups to help with later design combinations '''
# Get all the loadcases in the model
loadcases = db.getLoadsets("Loadcase")
# Envelope Settlement cases
settlement_env = db.createEnvelope("Settlement Env")
for lc in loadcases:
    if "Settlement" in lc.getName():
        settlement_env.addEntry(lc)
# Envelope +/- Temperature cases
temp_annual_env = db.createEnvelope("Temp Annual Env")
temp_annual_env.addEntry(temp_warm_loadcase)
temp_annual_env.addEntry(temp_cool_loadcase)
temp_daily_env = db.createEnvelope("Temp Daily Env")
temp_daily_env.addEntry(temp_warm_grad_loadcase)
temp_daily_env.addEntry(temp_cool_grad_loadcase)
# Then combine seasonal and daily variations. Note both max and min versions of the envelope should be included in the smart combination
temp_comb = db.createCombinationSmart("Temperature")
temp_comb.addEntry(1,0,temp_annual_env)
temp_comb.addEntry(1,0,temp_annual_env.getAssocLoadset())
temp_comb.addEntry(1,0,temp_daily_env)
temp_comb.addEntry(1,0,temp_daily_env.getAssocLoadset())
# Envelope Braking Loadcases
braking_env = db.createEnvelope("Braking Env")
for lc in loadcases:
    if "Braking" in lc.getName():
        braking_env.addEntry(lc)
# Envelope Horizontal Loadcases
braking_env = db.createEnvelope("Horizontal Env")
for lc in loadcases:
    if "Horizontal" in lc.getName():
        braking_env.addEntry(lc)

# Influence analysis

In [ ]:
# An influence analysis considers unit loads applied to a group of elements in order to generate influence surfaces.
# We need to specify the group of elements by assigning a search area
# Create a search area attribute
search_area_attr = db.createSearchArea("Deck Slab")
# Assign the search area attribute to the same surfaces
search_area_attr.assignTo("Surfaces", lusas.assignment().setAllDefaults())

In [ ]:
# Create a Direct Method Influence Analysis
dmi_analysis = db.createAnalysisDirectMethodInf("DMI")
# Specify the elements in the search area deck slab
dmi_analysis.setSearchArea("Deck Slab")
dmi_analysis.setSearchAssignType("area")
dmi_analysis.setIsGrillageType(1)
if dmi_grid:
    dmi_analysis.setGridCentreline("X")
    dmi_analysis.setGridWidth(carriageway_width)
dmi_analysis.setDoMatchNodes(dmi_grid)
#dmi_analysis.setIncludeMidSideNodes(False)
dmi_analysis.setSelectedResultsGroup("assignments")
dmi_analysis.createElements()

# Influence based on slice locations

In [ ]:
# Create an influence attribute which is assigned to slice locations
inf_attr = db.createDirectInfluence("Girder Major Bending")
inf_attr.setResultsTransformGlobal()
inf_attr.setInfluence("Beam/Shell Slice Resultants", "My")

# Solve the analyses

In [ ]:
# Solve each analysis and open all the available results files
if solve_analyses:
    db.getAnalysis("Analysis 1").solve(True)
    db.openAllResults(False)

In [ ]:
if solve_analyses:
    inf_attr.assignToBeamShellSlice("Girder 3", "Direct Method Influence Analysis 1")

In [ ]:
# lusas.view().insertValuesLayer()
# lusas.view().values().setResultsTransformNone()
# lusas.view().values().setResults("Reaction", "FZ")

Set the view to be perspective mode

In [ ]:
# lusas.view().setProjection(1)
# lusas.view().setIsometric()
lusas.view().setScaledToFit(True)
# lusas.view().geometry().colourByParentGroup(True)


Set the view to display the assigned loading in the attributes layers as well as the geometric "fleshing"

In [ ]:
lusas.view().insertAttributesLayer()
lusas.view().attributes().drawPatchByDefinition(True)
lusas.view().attributes().visualiseAll("Loading")
lusas.view().attributes().visualiseAll("Geometric")
#lusas.view().attributes().visualiseAllTransparent("Geometric")

if solve_analyses:
    lusas.view().insertDiagramsLayer()
    lusas.view().diagrams().setResultsTransformNone()
    lusas.view().diagrams().setResults("Beam/Shell Slice Resultants", "My")
    lusas.view().diagrams().setLocation("BeamShellSlice")
    lusas.view().diagrams().setAssocOption("Single")

# dont show beam shell slice definitions
lusas.view().utilities().setBeamShellSliceOptions(False, False, False, False)
